### 모듈

In [126]:
import numpy as np
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

### 데이터 불러오기

In [136]:
filenames = [
    "labeled_bond.csv",
    "labeled_economy.csv",
    "labeled_edaily.csv",
    "labeled_hk.csv",
    "labeled_infomax.csv",
    "labeled_mpb.csv"
]

df_list = []
for file in filenames:
    temp_df = pd.read_csv(file)
    df_list.append(temp_df)

# 모든 데이터가 합쳐진 최종 데이터프레임
df = pd.concat(df_list, ignore_index=True)

print(df.head())
print(df.shape)

         date                                        ngram_label
0  2024-11-29  [(('한국',), 89, -1), (('미국',), 254, -1), (('제조업...
1  2024-11-28  [(('트럼프',), 51, -1), (('관세',), 43, -1), (('인상'...
2  2024-11-27                                                 []
3  2024-11-26  [(('ETF',), 24, 0), (('자금',), 18, 0), (('동향',)...
4  2024-11-25  [(('한국',), 37, 0), (('미국',), 99, 0), (('시장',),...
(17403, 2)


### NBC 모델링

In [137]:
def parse_ngram_list(x):
    return ast.literal_eval(x)

df['ngram_list'] = df['ngram_label'].apply(parse_ngram_list)

In [138]:
def create_document_and_target(ngram_list):
    if not ngram_list:
        return "", None
    
    # 초기화
    tokens = []
    target = None

    for item in ngram_list:
        token_tuple, freq, lab = item
        # item이 (('금리',), 13, 0) 형태일 때,
        # item[0] = 토큰 튜플
        # item[1] = 빈도수(freq)
        # item[2] = 라벨(0이면 dovish, 1이면 hawkish)

        # 튜플을 문자열로 만들어 합친다.
        token_str = "_".join(token_tuple)
        if token_str not in tokens:
            tokens.append(token_str)

        if lab == 1:
            target = "hawkish"
        elif lab == -1:
            target = "dovish"

    document = " ".join(tokens) if tokens else ""
    return document, target

In [139]:
df[['document', 'target']] = df['ngram_list'].apply(lambda x: pd.Series(create_document_and_target(x)))
df = df.dropna(subset=['document', 'target'])

### 학습

In [140]:
# 피처(X)와 타겟(y) 준비
X = df['document']
y = df['target']

# 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# CountVectorizer로 문서 벡터화
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# 나이브 베이즈 분류기(MultinomialNB) 학습
clf = MultinomialNB()
clf.fit(X_train_vec, y_train)

# 테스트 데이터 예측 및 평가
y_pred = clf.predict(X_test_vec)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      dovish       0.93      0.76      0.84       130
     hawkish       0.79      0.93      0.86       123

    accuracy                           0.85       253
   macro avg       0.86      0.85      0.85       253
weighted avg       0.86      0.85      0.84       253



In [141]:
feature_names = vectorizer.get_feature_names_out()
probs = np.exp(clf.feature_log_prob_)
print("클래스 순서:", clf.classes_)

클래스 순서: ['dovish' 'hawkish']


In [142]:
sentiment_score = probs[1] - probs[0]

sentiment_df = pd.DataFrame({
    'feature': feature_names,
    'dovish_prob': probs[0],
    'hawkish_prob': probs[1],
    'sentiment_score': sentiment_score
})

print("Hawkish 쪽 상위 단어:")
print(sentiment_df.sort_values('sentiment_score', ascending=False).head(10))
print("\nDovish 쪽 상위 단어:")
print(sentiment_df.sort_values('sentiment_score', ascending=True).head(10))

Hawkish 쪽 상위 단어:
     feature  dovish_prob  hawkish_prob  sentiment_score
4216      인상     0.004011      0.011673         0.007662
997    금리_인상     0.002534      0.008755         0.006220
4273   인플레이션     0.001203      0.006859         0.005656
2023      물가     0.003555      0.008577         0.005022
1274      긴축     0.000128      0.003941         0.003813
114       가격     0.003209      0.006573         0.003364
5677      침체     0.000547      0.003695         0.003149
1310      내년     0.002279      0.004909         0.002630
1230      기준     0.003081      0.005645         0.002564
3726      연준     0.001841      0.004336         0.002495

Dovish 쪽 상위 단어:
     feature  dovish_prob  hawkish_prob  sentiment_score
4284      인하     0.008131      0.001350        -0.006781
1017   금리_인하     0.004923      0.000518        -0.004404
5115      중국     0.007621      0.005632        -0.001989
970       금리     0.015661      0.013705        -0.001957
4328      일본     0.003373      0.001432        -0.0019

In [143]:
# target 별로 모든 document를 하나의 텍스트로 합치기
grouped_docs = df.groupby('target')['document'].apply(lambda docs: " ".join(docs)).reset_index()

In [144]:
for idx, row in grouped_docs.iterrows():
    print(f"{row['target']} : {row['document'][:200]}...")

dovish : 한국 미국 제조업 경기 수출 불확실성 상승 지수 SP 생산 감소 추세 지속 정부 기업 정책 기대 내년 고용 서비스업 임금 상승률 둔화 확대 발표 ISM PMI 흐름 필요 국내 트럼프 수입 중국 증가 크 우려 PCE 확인 소비 연말 판단 가운데 매크로 이벤트 비농업 신규 실업률 주요 유로존 기준 하락 속도 경제 독일 금리 인하 관련 인플레이션 압력 서비스 소...
hawkish : 내용 작성 확인 투자 판단 증권 물가 상승 미국 인플레이션 높 상승률 평균 경제 수요 한국 유지 지속 발표 국가 이벤트 대비 분기 전월 에너지 제외 비 수입 지수 소매 판매 가스 생산 주택 매크로 소매_판매 고용 소비자 fed 시장 가격 가중치 소비 데이터 기준 비중 서비스 하락 증가 주거비 크 주요 금리 인상 둔화 경기 유로존 소비자_물가 금리_인상 팀 ...


In [145]:
grouped_docs

,target,document
0,dovish,한국 미국 제조업 경기 수출 불확실성 상승 지수 SP 생산 감소 추세 지속 정부 기...
1,hawkish,내용 작성 확인 투자 판단 증권 물가 상승 미국 인플레이션 높 상승률 평균 경제 수...


In [147]:
dovish_str = grouped_docs.loc[grouped_docs['target'] == 'dovish', 'document'].values[0]
hawkish_str = grouped_docs.loc[grouped_docs['target'] == 'hawkish', 'document'].values[0]

dovish_words = dovish_str.split()
hawkish_words = hawkish_str.split()

max_len = max(len(dovish_words), len(hawkish_words))
dovish_words += [None] * (max_len - len(dovish_words))
hawkish_words += [None] * (max_len - len(hawkish_words))

df = pd.DataFrame({
    'dovish': dovish_words,
    'hawkish': hawkish_words
})

df.to_csv('word_dictionary.csv', index=False, encoding='utf-8-sig')

In [149]:
pd.read_csv('word_dictionary.csv').head(20)

,dovish,hawkish
0,한국,내용
1,미국,작성
2,제조업,확인
3,경기,투자
4,수출,판단
5,불확실성,증권
6,상승,물가
7,지수,상승
8,SP,미국
9,생산,인플레이션
